# Chap05 变形

In [1]:
import numpy as np
import pandas as pd

## 长宽表的变形
1. 长表：信息比较详细的表
2. 宽表：信息比较汇总的表
3. 信息等价，只是数值的呈现方式不同
### `pivot`长表变宽表的函数
1. 三要素：变形后的行索引`index`、需要转到列索引的列`columns`、列和行索引对应的数值`values`
2. 新生成表的列索引是**columns**对应列的唯一值，新表的行索引是**index**对应列的唯一值，**values**对应了想要展示的数值列
3. pivot进行变形操作需要满足唯一性的要求，即新表中的行索引对应的列和列索引对应的列的组合必须唯一
4. 三个参数允许被设置为列表
### `pivot_table`
1. 如果不满足唯一性条件，必须通过聚合操作使得相同行列组合对应的多个值变为一个值
2. `aggfunc`参数指定使用的聚合函数，也可以传入以序列为输入标量为输出的聚合函数实现自定义操作
3. 还具有**边际汇总的功能**，可以通过设置`margins=True`实现，其中边际的聚合方式与`aggfunc`中给出的聚合方法一致
### `melt`宽表变长表的函数
1. `id_vars`长表的类似id列名(来自宽表的列名)
2. `value_vars`宽表的值所在的列名(来自宽表的列名)
3. `var_name`宽表的值所在的列转化为长表后变成值，该列的列名(自定义)
4. `value_name`值的列名(自定义)

In [2]:
# 长表
pd.DataFrame({'Gender':['F','F','M','M'], 'Height':[163,160,175,180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
# 宽表
pd.DataFrame({'Height:F':[163,160], 'Height:M':[175,180]})

,Height:F,Height:M
0,163,175
1,160,180


In [4]:
df = pd.DataFrame({'Class': [1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [8]:
df.pivot(index=['Class','Name'], columns='Subject', values='Grade')

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,75
2,Si Li,90,85


In [2]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [3]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [4]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x:x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [5]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


In [8]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80,90],
                   'Math':[80,75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [9]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese','Math'],
                    var_name = 'Subject',
                    value_name='Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75
